# Day 31: Model Serialization Security

In this lab, we will tackle **MLOps Security** by building a scanner for malicious pickle files.
Pickle files are often used to distribute models (e.g., PyTorch models are often pickled), but they are insecure by design.

In [ ]:
import sys
import os
import pickle

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.security.serialization import PickleScanner

## 1. Create a Benign Pickle

First, we create a safe pickle file containing standard data structures.

In [ ]:
safe_data = {"weights": [0.1, 0.5, -0.2], "bias": 0.0}
safe_pickle = pickle.dumps(safe_data)

scanner = PickleScanner()
issues = scanner.scan(safe_pickle)

print(f"Safe Pickle Issues: {issues}")

## 2. Create a Malicious Pickle

We will craft a pickle that attempts to run a command (e.g., `os.system('echo HACKED')`) when loaded.
**NOTE**: We will NOT load this pickle using `pickle.load()` to avoid running the exploit. We will only scan it.

In [ ]:
import os

class Malicious:
    def __reduce__(self):
        # This tells pickle how to reconstruct the object.
        # It says: call os.system("echo HACKED")
        return (os.system, ('echo HACKED',))

malicious_data = Malicious()
malicious_pickle = pickle.dumps(malicious_data)

print("Malicious pickle created (not loaded).")

## 3. Scan the Malicious Pickle

Our scanner should detect the usage of `os.system`.

In [ ]:
issues = scanner.scan(malicious_pickle)
print(f"Malicious Pickle Issues: {issues}")

if issues:
    print("!!! BLOCKING FILE LOAD !!!")
else:
    print("File passed scan (this is bad if it was actually malicious).")